In [1]:
import os
import sys
import math
import sklearn
import matplotlib


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import seaborn as sns
import statsmodels.api as sm


from tensorflow import keras
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_boston
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from sklearn.linear_model import RidgeCV, LassoCV, Ridge, Lasso
from sklearn.utils.validation import column_or_1d
from sklearn.model_selection import train_test_split

In [2]:
CSV_PATH = "complete-frame.csv"
CSV_MINER_PATH = "testminereffectiveness.csv"

In [3]:
def silent_evaluation(model, x_test, y_test):
    f = open('/dev/null', 'w')
    regular_stdout = sys.stdout
    sys.stdout = f
    test_loss, test_acc = model.evaluate(x_test, y_test)
    sys.stdout = regular_stdout
    print('Model Accuracy: {}'.format(test_acc))
    
    
def split_data(train_x, train_y, training=0.70, validation=0.5):
    train_size = training

    train_cnt = math.floor(train_x.shape[0] * train_size)
    x_train = train_x.iloc[0:train_cnt].values
    y_train = train_y.iloc[0:train_cnt].values
    x_test = train_x.iloc[train_cnt:]
    y_test = train_y.iloc[train_cnt:]

    division = validation

    train_cnt = math.floor(x_test.shape[0] * division)
    x_validate = x_test.iloc[0:train_cnt].values
    y_validate = y_test.iloc[0:train_cnt].values
    x_test = x_test.iloc[train_cnt:].values
    y_test = y_test.iloc[train_cnt:].values

    return x_train, y_train, x_test, y_test, x_validate, y_validate


def plot_graphs(history):
    # Plot training & validation accuracy values
    plt.plot(history.history['accuracy'])
    #plt.plot(history.history['val_accuracy'])
    plt.title('Model accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')
    plt.show()

    # Plot training & validation loss values
    plt.plot(history.history['loss'])
    #plt.plot(history.history['val_loss'])
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')
    plt.show()

In [4]:
def label_rename (row):
    return row['path_test'].split('/')[len(row['path_test'].split('/')) - 1].split('.')[0]

def load_frame():
    frame1 = pd.read_csv(CSV_PATH, sep=",")
    frame1 = frame1.sample(frac=1).reset_index(drop=True)
    frame1['TestClassName'] = frame1.apply(lambda row: label_rename(row), axis=1)
    frame2 = pd.read_csv(CSV_MINER_PATH, sep=',')
    frame = pd.merge(frame1, frame2, on='TestClassName')
    frame = frame.drop(['project', 'module', 'path_test','test_name','path_src',
                        'class_name','TestClassName','commit','Nº','Project'], axis=1)
    frame = frame.sample(frac=1).reset_index(drop=True)
    frame = frame.dropna()

    return frame


def load_static_frame():
    frame1 = pd.read_csv(CSV_PATH, sep=",")
    frame1 = frame1.sample(frac=1).reset_index(drop=True)
    frame1['TestClassName'] = frame1.apply(lambda row: label_rename(row), axis=1)
    frame2 = pd.read_csv(CSV_MINER_PATH, sep=',')
    frame = pd.merge(frame1, frame2, on='TestClassName')
    frame = frame.drop(['project', 'module', 'path_test','test_name','path_src',
                        'class_name','TestClassName','commit','Nº','Project'], axis=1)
    frame = frame.sample(frac=1).reset_index(drop=True)
    frame = frame.dropna()

    return frame


def load_quartile(frame):
    low, high = frame.mutation.quantile([0.25,0.75])
    frame_low = frame.query('mutation<{low}'.format(low=low))
    frame_high = frame.query('mutation>{high}'.format(high=high))
    frame_low['mutation'] = 0
    frame_high['mutation'] = 1
    frame = pd.concat([frame_low, frame_high], ignore_index=True)
    frame = frame.sample(frac=1).reset_index(drop=True)
    return frame;

def load_meaningful_subset(frame):
    columns = [frame.no_mutations,
                         frame.line_coverage,
                         frame.csm_FE,
                         frame.CONNECTIVITY_prod,
                         frame.CONNECTIVITY_test,
                         frame.isEagerTest,
                         frame.LOC_prod, frame.LOC_test, frame.WMC_prod,
                         frame.LCOM1_prod, frame.LCOM2_prod,
                         frame.LCOM4_prod, frame.McCABE_prod,
                         frame.RFC_prod, frame.MPC_prod,
                         frame.RFC_test, frame.MPC_test,
                         frame.LCOM1_test, frame.LCOM2_test,
                         frame.LCOM4_test, frame.LCC_test,
                         frame.LCC_test, frame.WMC_test,
                         frame.McCABE_test, frame.NOP_prod]
    
    data_x = pd.concat(columns, axis = 1).round(2)
    data_y = pd.concat([frame.mutation], axis = 1)
    return data_x, data_y, len(columns)


def load_meaningful_subset_static(frame):
    columns = [frame.no_mutations,
                         frame.csm_FE,
                         frame.CONNECTIVITY_prod,
                         frame.CONNECTIVITY_test,
                         frame.isEagerTest,
                         frame.LOC_prod, frame.LOC_test, frame.WMC_prod,
                         frame.LCOM1_prod, frame.LCOM2_prod,
                         frame.LCOM4_prod, frame.McCABE_prod,
                         frame.RFC_prod, frame.MPC_prod,
                         frame.RFC_test, frame.MPC_test,
                         frame.LCOM1_test, frame.LCOM2_test,
                         frame.LCOM4_test, frame.LCC_test,
                         frame.LCC_test, frame.WMC_test,
                         frame.McCABE_test, frame.NOP_prod]
    
    data_x = pd.concat(columns, axis = 1).round(2)
    data_y = pd.concat([frame.mutation], axis = 1)
    return data_x, data_y, len(columns)

def load_meaningful_subset_2(frame):
    #columns = [frame.line_coverage, frame.isAssertionRoulette, frame.isMysteryGuest,
    #   frame.isResourceOptimism, frame.isForTestersOnly, frame.COH_prod, frame.BUSWEIMER_prod,
    #   frame.BUSWEIMER_test, frame.csm_LM, frame.prod_readability]
    
    [frame.line_coverage,
    frame.COH_prod, frame.BUSWEIMER_prod, frame.csm_MC,
       frame.prod_readability, frame.prod_readability]
    
    data_x = pd.concat(columns, axis = 1).round(2)
    data_y = pd.concat([frame.mutation], axis = 1)
    return data_x, data_y, len(columns)




def load_all_data(frame):
    columns = [frame.no_mutations, frame.line_coverage, frame.isAssertionRoulette, frame.isEagerTest, frame.isLazyTest,
frame.isMysteryGuest, frame.isSensitiveEquality, frame.isResourceOptimism, frame.isForTestersOnly,
frame.isIndirectTesting, frame.LOC_prod, frame.HALSTEAD_prod, frame.RFC_prod, frame.CBO_prod, frame.MPC_prod, frame.IFC_prod, frame.DAC_prod,frame.DAC2_prod, frame.LCOM1_prod, frame.LCOM2_prod,
frame.LCOM3_prod, frame.LCOM4_prod, frame.CONNECTIVITY_prod, frame.LCOM5_prod, frame.COH_prod, frame.TCC_prod,
frame.LCC_prod, frame.ICH_prod, frame.WMC_prod, frame.NOA_prod, frame.NOPA_prod, frame.NOP_prod,
frame.McCABE_prod, frame.BUSWEIMER_prod, frame.LOC_test, frame.HALSTEAD_test, frame.RFC_test, frame.CBO_test,
frame.MPC_test, frame.IFC_test, frame.DAC_test, frame.DAC2_test, frame.LCOM1_test, frame.LCOM2_test,
frame.LCOM3_test, frame.LCOM4_test, frame.CONNECTIVITY_test, frame.LCOM5_test, frame.COH_test, frame.TCC_test,
frame.LCC_test, frame.ICH_test, frame.WMC_test, frame.NOA_test, frame.NOPA_test, frame.NOP_test, frame.McCABE_test,
frame.BUSWEIMER_test, frame.csm_CDSBP, frame.csm_CC, frame.csm_FD, frame.csm_Blob, frame.csm_SC, frame.csm_MC,
frame.csm_LM, frame.csm_FE, frame.prod_readability, frame.test_readability]
    data_x = pd.concat(columns, axis = 1).round(2)
    data_y = pd.concat([frame.mutation], axis = 1)
    return data_x, data_y, len(columns)
    

def load_all_data_with_mine(frame):
    columns = [frame.no_mutations, frame.line_coverage, frame.isAssertionRoulette, frame.isEagerTest, frame.isLazyTest,
frame.isMysteryGuest, frame.isSensitiveEquality, frame.isResourceOptimism, frame.isForTestersOnly,
frame.isIndirectTesting, frame.LOC_prod, frame.HALSTEAD_prod, frame.RFC_prod, frame.CBO_prod, frame.MPC_prod, frame.IFC_prod, frame.DAC_prod,frame.DAC2_prod, frame.LCOM1_prod, frame.LCOM2_prod,
frame.LCOM3_prod, frame.LCOM4_prod, frame.CONNECTIVITY_prod, frame.LCOM5_prod, frame.COH_prod, frame.TCC_prod,
frame.LCC_prod, frame.ICH_prod, frame.WMC_prod, frame.NOA_prod, frame.NOPA_prod, frame.NOP_prod,
frame.McCABE_prod, frame.BUSWEIMER_prod, frame.LOC_test, frame.HALSTEAD_test, frame.RFC_test, frame.CBO_test,
frame.MPC_test, frame.IFC_test, frame.DAC_test, frame.DAC2_test, frame.LCOM1_test, frame.LCOM2_test,
frame.LCOM3_test, frame.LCOM4_test, frame.CONNECTIVITY_test, frame.LCOM5_test, frame.COH_test, frame.TCC_test,
frame.LCC_test, frame.ICH_test, frame.WMC_test, frame.NOA_test, frame.NOPA_test, frame.NOP_test, frame.McCABE_test,
frame.BUSWEIMER_test, frame.csm_CDSBP, frame.csm_CC, frame.csm_FD, frame.csm_Blob, frame.csm_SC, frame.csm_MC,
frame.csm_LM, frame.csm_FE, frame.prod_readability, frame.test_readability,frame.Assrtions, frame.Conditions,frame.TryCatch, frame.Loop,frame.Hamcrest,frame.Mockito,
           frame.BadApi,frame.LOC,frame.Expressions, frame.Depth, frame.Vocabulary,
           frame.Understandability,frame.BodySize, frame.Dexterity, frame.NonWhiteCharacters]
    
    data_x = pd.concat(columns, axis = 1).round(2)
    data_y = pd.concat([frame.mutation], axis = 1)
    return data_x, data_y, len(columns)


def load_all_data_static(frame):
    columns = [frame.no_mutations, frame.isAssertionRoulette, frame.isEagerTest, frame.isLazyTest,
frame.isMysteryGuest, frame.isSensitiveEquality, frame.isResourceOptimism, frame.isForTestersOnly,
frame.isIndirectTesting, frame.LOC_prod, frame.HALSTEAD_prod, frame.RFC_prod, frame.CBO_prod, frame.MPC_prod, frame.IFC_prod, frame.DAC_prod,frame.DAC2_prod, frame.LCOM1_prod, frame.LCOM2_prod,
frame.LCOM3_prod, frame.LCOM4_prod, frame.CONNECTIVITY_prod, frame.LCOM5_prod, frame.COH_prod, frame.TCC_prod,
frame.LCC_prod, frame.ICH_prod, frame.WMC_prod, frame.NOA_prod, frame.NOPA_prod, frame.NOP_prod,
frame.McCABE_prod, frame.BUSWEIMER_prod, frame.LOC_test, frame.HALSTEAD_test, frame.RFC_test, frame.CBO_test,
frame.MPC_test, frame.IFC_test, frame.DAC_test, frame.DAC2_test, frame.LCOM1_test, frame.LCOM2_test,
frame.LCOM3_test, frame.LCOM4_test, frame.CONNECTIVITY_test, frame.LCOM5_test, frame.COH_test, frame.TCC_test,
frame.LCC_test, frame.ICH_test, frame.WMC_test, frame.NOA_test, frame.NOPA_test, frame.NOP_test, frame.McCABE_test,
frame.BUSWEIMER_test, frame.csm_CDSBP, frame.csm_CC, frame.csm_FD, frame.csm_Blob, frame.csm_SC, frame.csm_MC,
frame.csm_LM, frame.csm_FE, frame.prod_readability, frame.test_readability]
    data_x = pd.concat(columns, axis = 1).round(2)
    data_y = pd.concat([frame.mutation], axis = 1)
    return data_x, data_y, len(columns)
    

def load_all_data_with_mine_static(frame):
    columns = [frame.no_mutations, frame.isAssertionRoulette, frame.isEagerTest, frame.isLazyTest,
frame.isMysteryGuest, frame.isSensitiveEquality, frame.isResourceOptimism, frame.isForTestersOnly,
frame.isIndirectTesting, frame.LOC_prod, frame.HALSTEAD_prod, frame.RFC_prod, frame.CBO_prod, frame.MPC_prod, frame.IFC_prod, frame.DAC_prod,frame.DAC2_prod, frame.LCOM1_prod, frame.LCOM2_prod,
frame.LCOM3_prod, frame.LCOM4_prod, frame.CONNECTIVITY_prod, frame.LCOM5_prod, frame.COH_prod, frame.TCC_prod,
frame.LCC_prod, frame.ICH_prod, frame.WMC_prod, frame.NOA_prod, frame.NOPA_prod, frame.NOP_prod,
frame.McCABE_prod, frame.BUSWEIMER_prod, frame.LOC_test, frame.HALSTEAD_test, frame.RFC_test, frame.CBO_test,
frame.MPC_test, frame.IFC_test, frame.DAC_test, frame.DAC2_test, frame.LCOM1_test, frame.LCOM2_test,
frame.LCOM3_test, frame.LCOM4_test, frame.CONNECTIVITY_test, frame.LCOM5_test, frame.COH_test, frame.TCC_test,
frame.LCC_test, frame.ICH_test, frame.WMC_test, frame.NOA_test, frame.NOPA_test, frame.NOP_test, frame.McCABE_test,
frame.BUSWEIMER_test, frame.csm_CDSBP, frame.csm_CC, frame.csm_FD, frame.csm_Blob, frame.csm_SC, frame.csm_MC,
frame.csm_LM, frame.csm_FE, frame.prod_readability, frame.test_readability,frame.Assrtions, frame.Conditions,frame.TryCatch, frame.Loop,frame.Hamcrest,frame.Mockito,
           frame.BadApi,frame.LOC,frame.Expressions, frame.Depth, frame.Vocabulary,
           frame.Understandability,frame.BodySize, frame.Dexterity, frame.NonWhiteCharacters]
    
    data_x = pd.concat(columns, axis = 1).round(2)
    data_y = pd.concat([frame.mutation], axis = 1)
    return data_x, data_y, len(columns)

def load_mine(frame):
    columns = [frame.Assrtions, frame.Conditions,frame.TryCatch, frame.Loop,frame.Hamcrest,frame.Mockito,
           frame.BadApi,frame.LOC,frame.Expressions, frame.Depth, frame.Vocabulary,
           frame.Understandability,frame.BodySize, frame.Dexterity, frame.NonWhiteCharacters, frame.mutation]
    data_x = pd.concat(columns, axis = 1).round(2)
    data_y = pd.concat([frame.mutation], axis = 1)
    return data_x, data_y, len(columns)

In [5]:
def median(data_x, data_y, frame):
    labels = [1,2]
    bins = [0,frame.mutation.median(),1]
    frame['mutation_bins'] = pd.cut(frame.mutation, bins=bins, labels = labels, include_lowest=True)
    data_y = pd.concat([frame.mutation_bins], axis = 1)
    return split_data(data_x, data_y)

In [40]:
#Dropout + Early Stopping
#All data with mine (Static + mine)


from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler


#pa = [0,0.02,0.04,0.05,0.11,0.15,0.17,0.19,0.20,0.21,0.23,0.25,0.28,0.33,0.38,0.44,0.50,0.55,0.60,0.65,0.70,0.75,0.80,0.85,0.9,0.95,0.97]
pa = [0.20,0.21,0.23,0.25,0.28,0.33]

for p in pa:

    import tensorflow

    from numpy.random import seed
    seed(1)
    tensorflow.random.set_seed(2)


    frame = load_frame()
    frame = load_quartile(frame)

    data_x, data_y, number_of_features = load_all_data_with_mine_static(frame)

    data_x = data_x.values
    data_y = data_y.values
    
    scaler = StandardScaler()
    scaler.fit(data_x)
    data_x = scaler.transform(data_x)
       
    seed = 7
    np.random.seed(seed)
    kfold = StratifiedKFold(n_splits=10, shuffle=False, random_state=seed)
    cvscores = []
    overscores = []

    for train, test in kfold.split(data_x, data_y):

        model = keras.Sequential()

        model.add(keras.layers.Dropout(p, input_shape=(number_of_features,)))
        model.add(keras.layers.Dense(40, activation='relu'))
        model.add(keras.layers.Dense(20, activation='relu'))
        model.add(keras.layers.Dense(2, activation='softmax'))

        model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy',keras.metrics.Precision()])
        
             #, keras.metrics.Precision(), keras.metrics.Recall(), 
                      #keras.metrics.AUC()])

        early_stopping_monitor = keras.callbacks.EarlyStopping(monitor='accuracy', min_delta=0.0003, patience=10, verbose=1, mode='max', restore_best_weights=True)

        history = model.fit(data_x[train], data_y[train], epochs=2000, verbose=0,
                            callbacks=[early_stopping_monitor])
        
        scores = model.evaluate(data_x[test], data_y[test], verbose=0)
        overfit = model.evaluate(data_x[train], data_y[train], verbose=0)
        print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
        print("Overfit: %.2f%%" % (overfit[1]*100))
        #print("-------------------------------------------")

        cvscores.append(scores[1] * 100)
        overscores.append(overfit[1]*100)
    print("-------------------------------------------")
    print("The result, with a Dropout of {} is:".format(p))
    print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))
    print("And overfit of %.2f%% (+/- %.2f%%)" % (np.mean(overscores), np.std(overscores)))
    print("-------------------------------------------")

/Users/Dor/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/Dor/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/Dor/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  Fu

ValueError: Shapes (None, 2) and (None, 1) are incompatible

In [9]:
#Dropout + Early Stopping
#All original data (Static)

from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler


#pa = [0,0.02,0.04,0.05,0.11,0.15,0.17,0.19,0.20,0.21,0.23,0.25,0.28,0.33,0.38,0.44,0.50,0.55,0.60,0.65,0.70,0.75,0.80,0.85,0.9,0.95,0.97]
pa = [0.20,0.21,0.23,0.25,0.28,0.33]

for p in pa:

    import tensorflow

    from numpy.random import seed
    seed(1)
    tensorflow.random.set_seed(2)


    frame = load_frame()
    frame = load_quartile(frame)

    data_x, data_y, number_of_features = load_all_data_static(frame)

    data_x = data_x.values
    data_y = data_y.values
    
    scaler = StandardScaler()
    scaler.fit(data_x)
    data_x = scaler.transform(data_x)
       
    seed = 7
    np.random.seed(seed)
    kfold = StratifiedKFold(n_splits=10, shuffle=False, random_state=seed)
    cvscores = []
    overscores = []

    for train, test in kfold.split(data_x, data_y):

        model = keras.Sequential()

        model.add(keras.layers.Dropout(p, input_shape=(number_of_features,)))
        model.add(keras.layers.Dense(40, activation='relu'))
        model.add(keras.layers.Dense(20, activation='relu'))
        model.add(keras.layers.Dense(2, activation='softmax'))

        model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

        early_stopping_monitor = keras.callbacks.EarlyStopping(monitor='accuracy', min_delta=0.0003, patience=10, verbose=1, mode='max', restore_best_weights=True)

        history = model.fit(data_x[train], data_y[train], epochs=2000, verbose=0,
                            callbacks=[early_stopping_monitor])
        
        scores = model.evaluate(data_x[test], data_y[test], verbose=0)
        overfit = model.evaluate(data_x[train], data_y[train], verbose=0)
        print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
        print("Overfit: %.2f%%" % (overfit[1]*100))
        #print("-------------------------------------------")

        cvscores.append(scores[1] * 100)
        overscores.append(overfit[1]*100)
    print("-------------------------------------------")
    print("The result, with a Dropout of {} is:".format(p))
    print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))
    print("And overfit of %.2f%% (+/- %.2f%%)" % (np.mean(overscores), np.std(overscores)))
    print("-------------------------------------------")

/Users/Dor/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/Dor/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/Dor/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  Fu

Restoring model weights from the end of the best epoch.
Epoch 00043: early stopping
accuracy: 88.39%
Overfit: 94.93%
Restoring model weights from the end of the best epoch.
Epoch 00043: early stopping
accuracy: 86.61%
Overfit: 95.33%
Restoring model weights from the end of the best epoch.
Epoch 00044: early stopping
accuracy: 86.61%
Overfit: 95.03%
Restoring model weights from the end of the best epoch.
Epoch 00046: early stopping
accuracy: 91.07%
Overfit: 94.73%
Restoring model weights from the end of the best epoch.
Epoch 00061: early stopping
accuracy: 91.96%
Overfit: 96.52%
Restoring model weights from the end of the best epoch.
Epoch 00060: early stopping
accuracy: 91.07%
Overfit: 96.82%
Restoring model weights from the end of the best epoch.
Epoch 00061: early stopping
accuracy: 89.29%
Overfit: 97.32%
Restoring model weights from the end of the best epoch.
Epoch 00039: early stopping
accuracy: 89.29%
Overfit: 95.13%
Restoring model weights from the end of the best epoch.
Epoch 00

/Users/Dor/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/Dor/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/Dor/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  Fu

Restoring model weights from the end of the best epoch.
Epoch 00056: early stopping
accuracy: 90.18%
Overfit: 96.22%
Restoring model weights from the end of the best epoch.
Epoch 00098: early stopping
accuracy: 87.50%
Overfit: 98.51%
Restoring model weights from the end of the best epoch.
Epoch 00044: early stopping
accuracy: 85.71%
Overfit: 94.83%
Restoring model weights from the end of the best epoch.
Epoch 00050: early stopping
accuracy: 89.29%
Overfit: 95.73%
Restoring model weights from the end of the best epoch.
Epoch 00083: early stopping
accuracy: 92.86%
Overfit: 97.81%
Restoring model weights from the end of the best epoch.
Epoch 00075: early stopping
accuracy: 91.96%
Overfit: 98.01%
Restoring model weights from the end of the best epoch.
Epoch 00083: early stopping
accuracy: 89.29%
Overfit: 97.81%
Restoring model weights from the end of the best epoch.
Epoch 00039: early stopping
accuracy: 87.50%
Overfit: 94.93%
Restoring model weights from the end of the best epoch.
Epoch 00

/Users/Dor/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/Dor/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/Dor/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  Fu

...

Restoring model weights from the end of the best epoch.
Epoch 00043: early stopping
accuracy: 87.50%
Overfit: 94.14%
Restoring model weights from the end of the best epoch.
Epoch 00046: early stopping
accuracy: 86.61%
Overfit: 95.13%
Restoring model weights from the end of the best epoch.
Epoch 00044: early stopping
accuracy: 86.61%
Overfit: 94.43%
Restoring model weights from the end of the best epoch.
Epoch 00066: early stopping
accuracy: 91.96%
Overfit: 96.42%
Restoring model weights from the end of the best epoch.
Epoch 00060: early stopping
accuracy: 92.86%
Overfit: 95.63%
Restoring model weights from the end of the best epoch.
Epoch 00075: early stopping
accuracy: 91.96%
Overfit: 97.32%
Restoring model weights from the end of the best epoch.
Epoch 00107: early stopping
accuracy: 86.61%
Overfit: 98.11%
Restoring model weights from the end of the best epoch.
Epoch 00035: early stopping
accuracy: 87.50%
Overfit: 93.14%
Restoring model weights from the end of the best epoch.
Epoch 00

/Users/Dor/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/Dor/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/Dor/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  Fu

Restoring model weights from the end of the best epoch.
Epoch 00064: early stopping
accuracy: 90.18%
Overfit: 95.63%
Restoring model weights from the end of the best epoch.
Epoch 00041: early stopping
accuracy: 86.61%
Overfit: 93.14%
Restoring model weights from the end of the best epoch.
Epoch 00046: early stopping
accuracy: 85.71%
Overfit: 94.43%
Restoring model weights from the end of the best epoch.
Epoch 00049: early stopping
accuracy: 90.18%
Overfit: 93.14%
Restoring model weights from the end of the best epoch.
Epoch 00071: early stopping
accuracy: 92.86%
Overfit: 95.73%
Restoring model weights from the end of the best epoch.
Epoch 00057: early stopping
accuracy: 91.07%
Overfit: 95.53%
Restoring model weights from the end of the best epoch.
Epoch 00062: early stopping
accuracy: 85.71%
Overfit: 95.92%
Restoring model weights from the end of the best epoch.
Epoch 00050: early stopping
accuracy: 87.50%
Overfit: 94.63%
Restoring model weights from the end of the best epoch.
Epoch 00

/Users/Dor/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/Dor/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/Dor/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  Fu

Restoring model weights from the end of the best epoch.
Epoch 00043: early stopping
accuracy: 85.71%
Overfit: 92.25%
Restoring model weights from the end of the best epoch.
Epoch 00046: early stopping
accuracy: 87.50%
Overfit: 93.44%
Restoring model weights from the end of the best epoch.
Epoch 00044: early stopping
accuracy: 84.82%
Overfit: 92.35%
Restoring model weights from the end of the best epoch.
Epoch 00049: early stopping
accuracy: 89.29%
Overfit: 92.74%
Restoring model weights from the end of the best epoch.
Epoch 00032: early stopping
accuracy: 89.29%
Overfit: 90.76%
Restoring model weights from the end of the best epoch.
Epoch 00035: early stopping
accuracy: 86.61%
Overfit: 91.25%
Restoring model weights from the end of the best epoch.
Epoch 00071: early stopping
accuracy: 84.82%
Overfit: 94.93%
Restoring model weights from the end of the best epoch.
Epoch 00052: early stopping
accuracy: 86.61%
Overfit: 93.84%
Restoring model weights from the end of the best epoch.
Epoch 00

In [12]:
#Dropout + Early Stopping
#Early Stopping


from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler


pa = [0,0.02,0.04,0.05,0.11,0.15,0.17,0.19,0.20,0.21,0.23,0.25,0.28,0.33,0.38,0.44,0.50,0.55,0.60,0.65,0.70,0.75,0.80,0.85,0.9,0.95,0.97]
#pa = [0.20,0.21,0.23,0.25,0.28,0.33]

for p in pa:

    import tensorflow

    from numpy.random import seed
    seed(1)
    tensorflow.random.set_seed(2)


    frame = load_frame()
    frame = load_quartile(frame)

    data_x, data_y, number_of_features = load_meaningful_subset_static(frame)

    data_x = data_x.values
    data_y = data_y.values
    
    scaler = StandardScaler()
    scaler.fit(data_x)
    data_x = scaler.transform(data_x)
       
    seed = 7
    np.random.seed(seed)
    kfold = StratifiedKFold(n_splits=10, shuffle=False, random_state=seed)
    cvscores = []
    overscores = []

    for train, test in kfold.split(data_x, data_y):

        model = keras.Sequential()

        model.add(keras.layers.Dropout(p, input_shape=(number_of_features,)))
        model.add(keras.layers.Dense(number_of_features/2, activation='relu'))
        model.add(keras.layers.Dense(number_of_features/4, activation='relu'))
        model.add(keras.layers.Dense(2, activation='softmax'))

        model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

        early_stopping_monitor = keras.callbacks.EarlyStopping(monitor='accuracy', min_delta=0.0003, patience=10, verbose=1, mode='max', restore_best_weights=True)

        history = model.fit(data_x[train], data_y[train], epochs=2000, verbose=0,
                            callbacks=[early_stopping_monitor])
        
        scores = model.evaluate(data_x[test], data_y[test], verbose=0)
        overfit = model.evaluate(data_x[train], data_y[train], verbose=0)
        print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
        print("Overfit: %.2f%%" % (overfit[1]*100))
        #print("-------------------------------------------")

        cvscores.append(scores[1] * 100)
        overscores.append(overfit[1]*100)
    print("-------------------------------------------")
    print("The result, with a Dropout of {} is:".format(p))
    print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))
    print("And overfit of %.2f%% (+/- %.2f%%)" % (np.mean(overscores), np.std(overscores)))
    print("-------------------------------------------")

/Users/Dor/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/Dor/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/Dor/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  Fu

Restoring model weights from the end of the best epoch.
Epoch 00093: early stopping
accuracy: 86.61%
Overfit: 88.77%
Restoring model weights from the end of the best epoch.
Epoch 00050: early stopping
accuracy: 83.93%
Overfit: 84.89%
Restoring model weights from the end of the best epoch.
Epoch 00054: early stopping
accuracy: 83.04%
Overfit: 88.17%
Restoring model weights from the end of the best epoch.
Epoch 00054: early stopping
accuracy: 92.86%
Overfit: 85.39%
Restoring model weights from the end of the best epoch.
Epoch 00092: early stopping
accuracy: 85.71%
Overfit: 88.07%
Restoring model weights from the end of the best epoch.
Epoch 00095: early stopping
accuracy: 78.57%
Overfit: 88.27%
Restoring model weights from the end of the best epoch.
Epoch 00094: early stopping
accuracy: 80.36%
Overfit: 87.67%
Restoring model weights from the end of the best epoch.
Epoch 00055: early stopping
accuracy: 82.14%
Overfit: 84.89%
Restoring model weights from the end of the best epoch.
Epoch 00

/Users/Dor/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/Dor/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/Dor/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  Fu

Restoring model weights from the end of the best epoch.
Epoch 00040: early stopping
accuracy: 84.82%
Overfit: 84.59%
Restoring model weights from the end of the best epoch.
Epoch 00050: early stopping
accuracy: 83.93%
Overfit: 85.09%
Restoring model weights from the end of the best epoch.
Epoch 00050: early stopping
accuracy: 84.82%
Overfit: 86.18%
Restoring model weights from the end of the best epoch.
Epoch 00033: early stopping
accuracy: 88.39%
Overfit: 83.70%
Restoring model weights from the end of the best epoch.
Epoch 00035: early stopping
accuracy: 88.39%
Overfit: 84.29%
Restoring model weights from the end of the best epoch.
Epoch 00053: early stopping
accuracy: 80.36%
Overfit: 86.98%
Restoring model weights from the end of the best epoch.
Epoch 00026: early stopping
accuracy: 73.21%
Overfit: 83.60%
Restoring model weights from the end of the best epoch.
Epoch 00042: early stopping
accuracy: 84.82%
Overfit: 84.10%
Restoring model weights from the end of the best epoch.
Epoch 00

/Users/Dor/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/Dor/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/Dor/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  Fu

...

Restoring model weights from the end of the best epoch.
Epoch 00038: early stopping
accuracy: 82.14%
Overfit: 78.33%
Restoring model weights from the end of the best epoch.
Epoch 00024: early stopping
accuracy: 77.68%
Overfit: 75.25%
Restoring model weights from the end of the best epoch.
Epoch 00038: early stopping
accuracy: 71.43%
Overfit: 73.96%
Restoring model weights from the end of the best epoch.
Epoch 00035: early stopping
accuracy: 77.68%
Overfit: 74.06%
Restoring model weights from the end of the best epoch.
Epoch 00035: early stopping
accuracy: 83.93%
Overfit: 76.44%
Restoring model weights from the end of the best epoch.
Epoch 00024: early stopping
accuracy: 80.36%
Overfit: 78.13%
Restoring model weights from the end of the best epoch.
Epoch 00040: early stopping
accuracy: 68.75%
Overfit: 78.63%
Restoring model weights from the end of the best epoch.
Epoch 00052: early stopping
accuracy: 83.04%
Overfit: 77.63%
Restoring model weights from the end of the best epoch.
Epoch 00

/Users/Dor/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/Dor/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/Dor/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  Fu

Restoring model weights from the end of the best epoch.
Epoch 00042: early stopping
accuracy: 71.43%
Overfit: 68.69%
Restoring model weights from the end of the best epoch.
Epoch 00024: early stopping
accuracy: 53.57%
Overfit: 58.65%
Restoring model weights from the end of the best epoch.
Epoch 00034: early stopping
accuracy: 69.64%
Overfit: 68.09%
Restoring model weights from the end of the best epoch.
Epoch 00033: early stopping
accuracy: 69.64%
Overfit: 69.18%
Restoring model weights from the end of the best epoch.
Epoch 00029: early stopping
accuracy: 73.21%
Overfit: 70.58%
Restoring model weights from the end of the best epoch.
Epoch 00029: early stopping
accuracy: 74.11%
Overfit: 76.74%
Restoring model weights from the end of the best epoch.
Epoch 00025: early stopping
accuracy: 63.39%
Overfit: 65.01%
Restoring model weights from the end of the best epoch.
Epoch 00037: early stopping
accuracy: 75.89%
Overfit: 73.86%
Restoring model weights from the end of the best epoch.
Epoch 00

/Users/Dor/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/Dor/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/Dor/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  Fu

Restoring model weights from the end of the best epoch.
Epoch 00033: early stopping
accuracy: 61.61%
Overfit: 62.33%
Restoring model weights from the end of the best epoch.
Epoch 00017: early stopping
accuracy: 67.86%
Overfit: 67.30%
Restoring model weights from the end of the best epoch.
Epoch 00015: early stopping
accuracy: 52.68%
Overfit: 52.29%
Restoring model weights from the end of the best epoch.
Epoch 00025: early stopping
accuracy: 58.93%
Overfit: 60.74%
Restoring model weights from the end of the best epoch.
Epoch 00025: early stopping
accuracy: 61.61%
Overfit: 58.65%
Restoring model weights from the end of the best epoch.
Epoch 00039: early stopping
accuracy: 79.46%
Overfit: 73.16%
Restoring model weights from the end of the best epoch.
Epoch 00025: early stopping
accuracy: 61.61%
Overfit: 57.65%
Restoring model weights from the end of the best epoch.
Epoch 00033: early stopping
accuracy: 69.64%
Overfit: 66.10%
Restoring model weights from the end of the best epoch.
Epoch 00

In [13]:
#Dropout + Early Stopping
#All data with mine (Dynamic)

from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler


#pa = [0,0.02,0.04,0.05,0.11,0.15,0.17,0.19,0.20,0.21,0.23,0.25,0.28,0.33,0.38,0.44,0.50,0.55,0.60,0.65,0.70,0.75,0.80,0.85,0.9,0.95,0.97]
pa = [0.20,0.21,0.23,0.25,0.28,0.33]

for p in pa:

    import tensorflow

    from numpy.random import seed
    seed(1)
    tensorflow.random.set_seed(2)


    frame = load_frame()
    frame = load_quartile(frame)

    data_x, data_y, number_of_features = load_all_data_with_mine(frame)

    data_x = data_x.values
    data_y = data_y.values
    
    scaler = StandardScaler()
    scaler.fit(data_x)
    data_x = scaler.transform(data_x)
       
    seed = 7
    np.random.seed(seed)
    kfold = StratifiedKFold(n_splits=10, shuffle=False, random_state=seed)
    cvscores = []
    overscores = []

    for train, test in kfold.split(data_x, data_y):

        model = keras.Sequential()

        model.add(keras.layers.Dropout(p, input_shape=(number_of_features,)))
        model.add(keras.layers.Dense(40, activation='relu'))
        model.add(keras.layers.Dense(20, activation='relu'))
        model.add(keras.layers.Dense(2, activation='softmax'))

        model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

        early_stopping_monitor = keras.callbacks.EarlyStopping(monitor='accuracy', min_delta=0.0003, patience=10, verbose=1, mode='max', restore_best_weights=True)

        history = model.fit(data_x[train], data_y[train], epochs=2000, verbose=0,
                            callbacks=[early_stopping_monitor])
        
        scores = model.evaluate(data_x[test], data_y[test], verbose=0)
        overfit = model.evaluate(data_x[train], data_y[train], verbose=0)
        print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
        print("Overfit: %.2f%%" % (overfit[1]*100))
        #print("-------------------------------------------")

        cvscores.append(scores[1] * 100)
        overscores.append(overfit[1]*100)
    print("-------------------------------------------")
    print("The result, with a Dropout of {} is:".format(p))
    print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))
    print("And overfit of %.2f%% (+/- %.2f%%)" % (np.mean(overscores), np.std(overscores)))
    print("-------------------------------------------")

/Users/Dor/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/Dor/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/Dor/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  Fu

Restoring model weights from the end of the best epoch.
Epoch 00037: early stopping
accuracy: 96.43%
Overfit: 98.41%
Restoring model weights from the end of the best epoch.
Epoch 00042: early stopping
accuracy: 94.64%
Overfit: 98.31%
Restoring model weights from the end of the best epoch.
Epoch 00042: early stopping
accuracy: 95.54%
Overfit: 98.41%
Restoring model weights from the end of the best epoch.
Epoch 00048: early stopping
accuracy: 92.86%
Overfit: 99.50%
Restoring model weights from the end of the best epoch.
Epoch 00041: early stopping
accuracy: 94.64%
Overfit: 98.31%
Restoring model weights from the end of the best epoch.
Epoch 00042: early stopping
accuracy: 94.64%
Overfit: 99.11%
Restoring model weights from the end of the best epoch.
Epoch 00041: early stopping
accuracy: 95.54%
Overfit: 98.11%
Restoring model weights from the end of the best epoch.
Epoch 00076: early stopping
accuracy: 92.86%
Overfit: 99.60%
Restoring model weights from the end of the best epoch.
Epoch 00

/Users/Dor/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/Dor/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/Dor/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  Fu

Restoring model weights from the end of the best epoch.
Epoch 00037: early stopping
accuracy: 97.32%
Overfit: 98.41%
Restoring model weights from the end of the best epoch.
Epoch 00042: early stopping
accuracy: 93.75%
Overfit: 98.01%
Restoring model weights from the end of the best epoch.
Epoch 00042: early stopping
accuracy: 96.43%
Overfit: 98.71%
Restoring model weights from the end of the best epoch.
Epoch 00048: early stopping
accuracy: 93.75%
Overfit: 99.50%
Restoring model weights from the end of the best epoch.
Epoch 00041: early stopping
accuracy: 92.86%
Overfit: 98.01%
Restoring model weights from the end of the best epoch.
Epoch 00026: early stopping
accuracy: 92.86%
Overfit: 96.62%
Restoring model weights from the end of the best epoch.
Epoch 00041: early stopping
accuracy: 95.54%
Overfit: 98.21%
Restoring model weights from the end of the best epoch.
Epoch 00041: early stopping
accuracy: 91.07%
Overfit: 98.11%
Restoring model weights from the end of the best epoch.
Epoch 00

/Users/Dor/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/Dor/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/Dor/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  Fu

...

Restoring model weights from the end of the best epoch.
Epoch 00037: early stopping
accuracy: 97.32%
Overfit: 98.21%
Restoring model weights from the end of the best epoch.
Epoch 00038: early stopping
accuracy: 94.64%
Overfit: 97.32%
Restoring model weights from the end of the best epoch.
Epoch 00042: early stopping
accuracy: 95.54%
Overfit: 98.11%
Restoring model weights from the end of the best epoch.
Epoch 00042: early stopping
accuracy: 92.86%
Overfit: 98.61%
Restoring model weights from the end of the best epoch.
Epoch 00041: early stopping
accuracy: 93.75%
Overfit: 97.51%
Restoring model weights from the end of the best epoch.
Epoch 00048: early stopping
accuracy: 94.64%
Overfit: 98.21%
Restoring model weights from the end of the best epoch.
Epoch 00041: early stopping
accuracy: 93.75%
Overfit: 97.81%
Restoring model weights from the end of the best epoch.
Epoch 00026: early stopping
accuracy: 90.18%
Overfit: 96.82%
Restoring model weights from the end of the best epoch.
Epoch 00

/Users/Dor/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/Dor/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/Dor/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  Fu

Restoring model weights from the end of the best epoch.
Epoch 00037: early stopping
accuracy: 96.43%
Overfit: 98.31%
Restoring model weights from the end of the best epoch.
Epoch 00031: early stopping
accuracy: 96.43%
Overfit: 97.12%
Restoring model weights from the end of the best epoch.
Epoch 00048: early stopping
accuracy: 95.54%
Overfit: 98.41%
Restoring model weights from the end of the best epoch.
Epoch 00048: early stopping
accuracy: 93.75%
Overfit: 98.81%
Restoring model weights from the end of the best epoch.
Epoch 00064: early stopping
accuracy: 95.54%
Overfit: 98.91%
Restoring model weights from the end of the best epoch.
Epoch 00055: early stopping
accuracy: 94.64%
Overfit: 98.71%
Restoring model weights from the end of the best epoch.
Epoch 00041: early stopping
accuracy: 95.54%
Overfit: 97.91%
Restoring model weights from the end of the best epoch.
Epoch 00054: early stopping
accuracy: 93.75%
Overfit: 98.41%
Restoring model weights from the end of the best epoch.
Epoch 00

/Users/Dor/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/Dor/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/Dor/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  Fu

Restoring model weights from the end of the best epoch.
Epoch 00031: early stopping
accuracy: 92.86%
Overfit: 96.42%
Restoring model weights from the end of the best epoch.
Epoch 00031: early stopping
accuracy: 96.43%
Overfit: 96.42%
Restoring model weights from the end of the best epoch.
Epoch 00053: early stopping
accuracy: 96.43%
Overfit: 98.51%
Restoring model weights from the end of the best epoch.
Epoch 00048: early stopping
accuracy: 92.86%
Overfit: 98.31%
Restoring model weights from the end of the best epoch.
Epoch 00041: early stopping
accuracy: 92.86%
Overfit: 97.02%
Restoring model weights from the end of the best epoch.
Epoch 00044: early stopping
accuracy: 93.75%
Overfit: 98.31%
Restoring model weights from the end of the best epoch.
Epoch 00026: early stopping
accuracy: 92.86%
Overfit: 96.02%
Restoring model weights from the end of the best epoch.
Epoch 00064: early stopping
accuracy: 92.86%
Overfit: 98.41%
Restoring model weights from the end of the best epoch.
Epoch 00

In [34]:

lst = [frame.no_mutations, frame.isAssertionRoulette, frame.isEagerTest, frame.isLazyTest,frame.isMysteryGuest, frame.isSensitiveEquality, frame.isResourceOptimism, frame.isForTestersOnly,
frame.isIndirectTesting, frame.LOC_prod, frame.HALSTEAD_prod, frame.RFC_prod, frame.CBO_prod, frame.MPC_prod, frame.IFC_prod, frame.DAC_prod,frame.DAC2_prod, frame.LCOM1_prod, frame.LCOM2_prod,
frame.LCOM3_prod, frame.LCOM4_prod, frame.CONNECTIVITY_prod, frame.LCOM5_prod, frame.COH_prod, frame.TCC_prod,
frame.LCC_prod, frame.ICH_prod, frame.WMC_prod, frame.NOA_prod, frame.NOPA_prod, frame.NOP_prod,
frame.McCABE_prod, frame.BUSWEIMER_prod, frame.LOC_test, frame.HALSTEAD_test, frame.RFC_test, frame.CBO_test,
frame.MPC_test, frame.IFC_test, frame.DAC_test, frame.DAC2_test, frame.LCOM1_test, frame.LCOM2_test,
frame.LCOM3_test, frame.LCOM4_test, frame.CONNECTIVITY_test, frame.LCOM5_test, frame.COH_test, frame.TCC_test,
frame.LCC_test, frame.ICH_test, frame.WMC_test, frame.NOA_test, frame.NOPA_test, frame.NOP_test, frame.McCABE_test,
frame.BUSWEIMER_test, frame.csm_CDSBP, frame.csm_CC, frame.csm_FD, frame.csm_Blob, frame.csm_SC, frame.csm_MC,
frame.csm_LM, frame.csm_FE, frame.prod_readability, frame.test_readability]
 
def load_exp(frame):
    columns = lst

    data_x = pd.concat(columns, axis = 1).round(2)
    data_y = pd.concat([frame.mutation], axis = 1)
    return data_x, data_y, len(columns)


import tensorflow


from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from numpy.random import seed
from sklearn.decomposition import FactorAnalysis


seed(1)
tensorflow.random.set_seed(2)


frame = load_frame()
frame = load_quartile(frame)


data_x, data_y, number_of_features = load_exp(frame)

feature_names = data_x.columns

data_x = data_x.values
data_y = data_y.values

scaler = StandardScaler()
scaler.fit(data_x)
data_x = scaler.transform(data_x)

from factor_analyzer.factor_analyzer import calculate_bartlett_sphericity
chi_square_value,p_value=calculate_bartlett_sphericity(data_x)
print(chi_square_value, p_value)

from factor_analyzer.factor_analyzer import calculate_kmo
kmo_all,kmo_model=calculate_kmo(data_x)

print(kmo_model)



nan nan
nan


/Users/Dor/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/Dor/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/Dor/anaconda3/lib/python3.6/site-packages/factor_analyzer/utils.py:91: RuntimeWarning: divide by zero encountered in true_divide
  x = (x - x.mean(0)) / x.std(0)
/Users/Dor/anaconda3/lib/python3.6/site-packages/factor_analyzer/utils.py:91: RuntimeWarning: invalid value